# Process 기반 병렬 프로그래밍
* Book - Python Prallel Programming Cookbook, Chapter 3.
* 다음 코드들은 **command-line**에서 실행해야 함.

##  프로세스 생성(Spawn a Process)
* object process 만들고,
* `start()` 메소드를 실행시킨 후,
* `join()` 메소드로 마무리

In [1]:
import multiprocessing

def foo(i):
    print('새로운 프로세스로 호출된 함수임: %s' % i)
    return

if __name__ == '__main__':
    Process_jobs = []
    for i in range(5):
        p = multiprocessing.Process(target=foo, args=(i,))
        Process_jobs.append(p)
        p.start()
        p.join()  # join()이 없으면 자식프로세스는 끝나지 않고 idle상태로 남음

새로운 프로세스로 호출된 함수임: 0
새로운 프로세스로 호출된 함수임: 1
새로운 프로세스로 호출된 함수임: 2
새로운 프로세스로 호출된 함수임: 3
새로운 프로세스로 호출된 함수임: 4


## 프로세스 이름 짓기

In [2]:
import multiprocessing
import time

def foo():
    name = multiprocessing.current_process().name
    print("시작: %s \n" % name)
    time.sleep(3)
    print("종료: %s \n" % name)
    
if __name__ == '__main__':
    process_with_name = multiprocessing.Process(name='foo_process', target=foo)
    process_with_name.daemon = True
    process_with_default_name = multiprocessing.Process(target=foo)
    process_with_name.start()
    process_with_default_name.start()
    process_with_name.join()
    process_with_default_name.join()

시작: foo_process 

시작: Process-7 

종료: foo_process 

종료: Process-7 



## 백그라운드로 프로세스 실행하기
* `daemon` 파라미터를 `True`로 설정
* 백그라운드 프로세스는 자식 프로세스를 생성할 수 없음

In [3]:
import multiprocessing
import time

def foo():
    name = multiprocessing.current_process().name
    print("시작: %s \n" % name)
    time.sleep(3)
    print("종료 %s \n" % name)

if __name__ == '__main__':
    background_process = multiprocessing.Process(name='background_process', target=foo)
    background_process.daemon = True
    No_background_process = multiprocessing.Process(name='No_background_process', target=foo)
    No_background_process.daemon = False

    background_process.start()
    No_background_process.start()
    background_process.join()
    No_background_process.join()

시작: background_process 

시작: No_background_process 

종료 background_process 
종료 No_background_process 




## 프로세스 죽이기
* `terminate()` 사용
* `is_alive()` 사용하여 검증

In [5]:
import multiprocessing
import time

def foo():
    print ('함수 시작')
    time.sleep(0.1)
    print ('함수 끝냄')

if __name__ == '__main__':
    p = multiprocessing.Process(target=foo)
    print ('실행하기 전 프로세스 is_alive():', p, p.is_alive())

    p.start()
    print ('프로세스 실행 중 is_alive():', p, p.is_alive())

    p.terminate()
    print ('프로세스 terminate() 후 is_alive():', p, p.is_alive())

    p.join()
    print ('프로세스 join() 후 is_alive():', p, p.is_alive())
    print ('프로세스 exit code:', p.exitcode)  # 0이면 에러 없음
                                               # 0보다 크면, 에러 존재하며 코드 끝냄
                                               # 0보다 작으면, -1*ExitCode 시그널로 죽임

실행하기 전 프로세스 is_alive(): <Process(Process-11, initial)> False
프로세스 실행 중 is_alive(): <Process(Process-11, started)> True
프로세스 terminate() 후 is_alive(): <Process(Process-11, started)> True
프로세스 join() 후 is_alive(): <Process(Process-11, stopped[SIGTERM])> False
프로세스 exit code: -15


## subclass로 프로세스 사용하기
* `Process` 클래스의 하위 클래스 정의
* `__init__(self [, args])` 재정의하여 인수 추가
* `run(self [, args])` 재정의하여 `Process`가 시작된 후 수행할 일 구현

In [6]:
import multiprocessing

class MyProcess(multiprocessing.Process):
    def run(self):
        print ('프로세스 내 run() 메소드: %s' % self.name)
        return

if __name__ == '__main__':
    jobs = []
    for i in range(5):
        p = MyProcess ()
        jobs.append(p)
        p.start()
        p.join()

프로세스 내 run() 메소드: MyProcess-12
프로세스 내 run() 메소드: MyProcess-13
프로세스 내 run() 메소드: MyProcess-14
프로세스 내 run() 메소드: MyProcess-15
프로세스 내 run() 메소드: MyProcess-16


## 프로세스들 간에 객체 교환
* queues
* pipes

In [7]:
import multiprocessing
import random
import time

class producer(multiprocessing.Process):
    def __init__(self, queue):
        multiprocessing.Process.__init__(self)
        self.queue = queue

    def run(self) :
        for i in range(10):
            item = random.randint(0, 256)
            self.queue.put(item)
            print ("생산 알림: {}가 아이템 {}를 큐에 넣음".format(self.name, item))
            time.sleep(1)
            print ("현재 큐에 들어있는 아이템 개수: %s" % self.queue.qsize())
            
class consumer(multiprocessing.Process):
    def __init__(self, queue):
        multiprocessing.Process.__init__(self)
        self.queue = queue

    def run(self):
        time.sleep(3)
        while True:
            if (self.queue.empty()):
                print("큐 비어있음")
                break
            else :
                time.sleep(2)
                item = self.queue.get()
                print ('소비 알림: {}가 아이템 {}를 큐에서 꺼냄'.format(self.name, item))
                time.sleep(1)

if __name__ == '__main__':
    queue = multiprocessing.Queue()
    process_producer = producer(queue)
    process_consumer = consumer(queue)
    process_producer.start()
    process_consumer.start()
    process_producer.join()
    process_consumer.join()

생산 알림: producer-17가 아이템 186를 큐에 넣음
현재 큐에 들어있는 아이템 개수: 1
생산 알림: producer-17가 아이템 26를 큐에 넣음
현재 큐에 들어있는 아이템 개수: 2
생산 알림: producer-17가 아이템 183를 큐에 넣음
현재 큐에 들어있는 아이템 개수: 3
생산 알림: producer-17가 아이템 29를 큐에 넣음
현재 큐에 들어있는 아이템 개수: 4
생산 알림: producer-17가 아이템 234를 큐에 넣음
소비 알림: consumer-18가 아이템 186를 큐에서 꺼냄
현재 큐에 들어있는 아이템 개수: 4
생산 알림: producer-17가 아이템 219를 큐에 넣음
현재 큐에 들어있는 아이템 개수: 5
생산 알림: producer-17가 아이템 46를 큐에 넣음
현재 큐에 들어있는 아이템 개수: 6
생산 알림: producer-17가 아이템 43를 큐에 넣음
소비 알림: consumer-18가 아이템 26를 큐에서 꺼냄
현재 큐에 들어있는 아이템 개수: 6
생산 알림: producer-17가 아이템 208를 큐에 넣음
현재 큐에 들어있는 아이템 개수: 7
생산 알림: producer-17가 아이템 33를 큐에 넣음
현재 큐에 들어있는 아이템 개수: 8
소비 알림: consumer-18가 아이템 183를 큐에서 꺼냄
소비 알림: consumer-18가 아이템 29를 큐에서 꺼냄
소비 알림: consumer-18가 아이템 234를 큐에서 꺼냄
소비 알림: consumer-18가 아이템 219를 큐에서 꺼냄
소비 알림: consumer-18가 아이템 46를 큐에서 꺼냄
소비 알림: consumer-18가 아이템 43를 큐에서 꺼냄
소비 알림: consumer-18가 아이템 208를 큐에서 꺼냄
소비 알림: consumer-18가 아이템 33를 큐에서 꺼냄
큐 비어있음


In [8]:
import multiprocessing

def create_items(pipe):
    output_pipe, _ = pipe
    for item in range(10):
        output_pipe.send(item)
    output_pipe.close()

def multiply_items(pipe_1, pipe_2):
    close, input_pipe = pipe_1
    close.close()
    output_pipe, _ = pipe_2
    try:
        while True:
            item = input_pipe.recv()
            output_pipe.send(item * item)
    except EOFError:
        output_pipe.close()

if __name__ == '__main__':
    #First process pipe with numbers from 0 to 9
    pipe_1 = multiprocessing.Pipe(True)
    process_pipe_1 = multiprocessing.Process(target=create_items, args=(pipe_1,))
    process_pipe_1.start()

    #second pipe,
    pipe_2 = multiprocessing.Pipe(True)
    process_pipe_2 = multiprocessing.Process(target=multiply_items, args=(pipe_1, pipe_2,))
    process_pipe_2.start()

    pipe_1[0].close()
    pipe_2[0].close()

    try:
        while True:
            print (pipe_2[1].recv())
    except EOFError:
        print("End")

0
1
4
9
16
25
36
49
64
81
End


## 프로세스 동기화(synchronization)
![Barrier](Python_Process_Parallelism-barrier.png)

In [10]:
import multiprocessing
from multiprocessing import Barrier, Lock, Process
from time import time
from datetime import datetime

def test_with_barrier(synchronizer, serializer):
    name = multiprocessing.current_process().name
    synchronizer.wait()
    now = time()
    with serializer:
        print("process %s ----> %s" % (name, datetime.fromtimestamp(now)))

def test_without_barrier():
    name = multiprocessing.current_process().name
    now = time()
    print("process %s ----> %s" % (name, datetime.fromtimestamp(now)))

if __name__ == '__main__':
    synchronizer = Barrier(2)  # 인수 2는 총 관리 프로세스가 2개임을 의미
    serializer = Lock()
    p1 = Process(name='p1 - test_with_barrier', target=test_with_barrier, args=(synchronizer, serializer))
    p2 = Process(name='p2 - test_with_barrier', target=test_with_barrier, args=(synchronizer, serializer))
    p3 = Process(name='p3 - test_without_barrier', target=test_without_barrier)
    p4 = Process(name='p4 - test_without_barrier', target=test_without_barrier)

    p1.start()
    p2.start()
    p3.start()
    p4.start()

    # p1과 p2는 같은 timestamp를 출력

    p1.join()
    p2.join()
    p3.join()
    p4.join()

process p1 - test_with_barrier ----> 2017-12-21 11:46:53.470492
process p3 - test_without_barrier ----> 2017-12-21 11:46:53.474489
process p2 - test_with_barrier ----> 2017-12-21 11:46:53.472623
process p4 - test_without_barrier ----> 2017-12-21 11:46:53.477442


## 프로세스들 간 상태 관리
* `Manager`는 공유 객체를 관리하는 서버 프로세스를 제어
* 모든 프로세스들이 변경된 공유 객체를 최신으로 유지

In [11]:
import multiprocessing

def worker(dictionary, key, item):
    dictionary[key] = item
    print("key=%i  value=%i" % (key, item))

if __name__ == '__main__':
    mgr = multiprocessing.Manager()
    dictionary = mgr.dict()
    jobs = [ multiprocessing.Process(target=worker, args=(dictionary, i, i*2)) for i in range(10)]
    for j in jobs:
        j.start()
    for j in jobs:
        j.join()
    print ('Results:', dictionary)

key=1  value=2
key=0  value=0
key=2  value=4
key=3  value=6
key=5  value=10
key=4  value=8
key=6  value=12
key=7  value=14
key=8  value=16
key=9  value=18
Results: {0: 0, 1: 2, 2: 4, 3: 6, 4: 8, 5: 10, 6: 12, 7: 14, 8: 16, 9: 18}


## 프로세스 풀 사용
* `Pool` 클래스는 다음 메소드 제공
  - `apply()` : 결과가 준비될 때까지 멈춤
  - `apply_async()` : 모든 자식 클래스들이 실행될 때까지 메인 스레드가 lock되지 않고 결과를 반환
  - `map()` : python `map()`의 병렬 버전. 결과가 준비될 때까지 멈춤.
  - `map_async()`

In [12]:
def function_square(data):
    result = data*data
    return result

if __name__ == '__main__':
    inputs = list(range(100))
    pool = multiprocessing.Pool(processes=4)
    pool_outputs = pool.map(function_square, inputs)
    pool.close()
    pool.join()
    print ('Pool :', pool_outputs)

Pool : [0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801]
